In [ ]:
import random
import numpy as np
from PIL import Image

# Constants
NUM_INDIVIDUALS = 50
NUM_GENERATIONS = 1000
NUM_RADICALS = 5

# Load the target line art and kanji radicals
target = Image.open("line_art.jpg").convert('L')
kanji_radicals = [Image.open(f"radical_{i}.jpg").convert('L') for i in range(NUM_RADICALS)]

# Initialization
def create_individual():
    return [(random.choice(kanji_radicals), random.randint(0, target.width), random.randint(0, target.height), random.uniform(0.5, 1.5)) for _ in range(NUM_RADICALS)]

population = [create_individual() for _ in range(NUM_INDIVIDUALS)]

# Fitness function
def fitness(individual):
    canvas = Image.new('L', target.size, color=255)
    for radical, x, y, size in individual:
        scaled_radical = radical.resize((int(radical.width * size), int(radical.height * size)))
        canvas.paste(scaled_radical, (x, y), scaled_radical)
    error = np.sum((np.array(canvas) - np.array(target))**2)
    return -error

# Selection, Crossover and Mutation
def crossover(parent1, parent2):
    split = random.randint(0, NUM_RADICALS-1)
    child = parent1[:split] + parent2[split:]
    return child

def mutate(individual):
    idx = random.randint(0, NUM_RADICALS-1)
    individual[idx] = (random.choice(kanji_radicals), random.randint(0, target.width), random.randint(0, target.height), random.uniform(0.5, 1.5))
    return individual

# Evolution
for generation in range(NUM_GENERATIONS):
    population.sort(key=fitness, reverse=True)
    new_population = population[:10]
    while len(new_population) < NUM_INDIVIDUALS:
        parent1 = random.choice(population[:10])
        parent2 = random.choice(population[:10])
        child = crossover(parent1, parent2)
        if random.random() < 0.1:
            child = mutate(child)
        new_population.append(child)
    population = new_population

# Display the best individual
best = population[0]
canvas = Image.new('L', target.size, color=255)
for radical, x, y, size in best:
    scaled_radical = radical.resize((int(radical.width * size), int(radical.height * size)))
    canvas.paste(scaled_radical, (x, y), scaled_radical)
canvas.show()
